In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd gdrive/My\ Drive/KKB-kaggle/bengaliai-cv19/notebooks

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/My Drive/KKB-kaggle/bengaliai-cv19/notebooks'
/content/gdrive/My Drive/KKB-kaggle/bengaliai-cv19/notebooks


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
!pip install efficientnet_pytorch

In [0]:
## 諸々の import

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
import cv2
# from tqdm.auto import tqdm
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import PIL
# from torchsummary import summary
import gc


dataset_dir = '../dataset'
# dataset_dir = '/kaggle/input/bengaliai-cv19'
train_df = pd.read_csv(dataset_dir + '/train.csv')
test_df = pd.read_csv(dataset_dir + '/test.csv')
class_map_df = pd.read_csv(dataset_dir + '/class_map.csv')
sample_sub_df = pd.read_csv(dataset_dir + '/sample_submission.csv')

model_dir = '../trained_models'
# model_dir = '/kaggle/input/models'

In [0]:
from preprocess import *
# from model.CNN import model
from model.efficientnet import model
# from model.resnet18 import model

model = model()
model = try_gpu(model)

criterion1 = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters())

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0


In [0]:
## Parameters

# resize後のサイズ
HEIGHT = 64
WIDTH = 64

# 画像を3次元にするかどうか（EfficientNetなどを使うときはTrue）
enable_3d = True

# True なら Cross Validation を実施する
# Kaggle に提出するときは False にしてください
do_validation = True

# True なら submission.csv を生成する
create_submission = False

val_perc = 0.2  # validation set の割合（クロスバリデーション）
start_epoch = 12
epochs = 5

In [0]:
## train関数、test関数

def train(model, epoch, train_loader):
    model.train()
    for i, data in enumerate(train_loader, 0):
        inputs, root_l, vowel_l, consonant_l = data
        inputs, root_l, vowel_l, consonant_l = Variable(inputs), Variable(root_l), Variable(vowel_l), Variable(consonant_l)
        inputs, root_l, vowel_l, consonant_l = try_gpu(inputs), try_gpu(root_l), try_gpu(vowel_l), try_gpu(consonant_l)
        optimizer.zero_grad()
        root_o, vowel_o, consonant_o = model(inputs)
        loss1 = criterion1(root_o, root_l)
        loss2 = criterion1(vowel_o, vowel_l)
        loss3 = criterion1(consonant_o, consonant_l)
        (loss1+loss2+loss3).backward()
        optimizer.step()
   

def test(model, test_loader):
    model.eval()

    size = len(test_loader.dataset)
    pred_r, pred_v, pred_c = np.zeros(size), np.zeros(size), np.zeros(size)
    true_r, true_v, true_c = np.zeros(size), np.zeros(size), np.zeros(size)
    index = 0
    
    for data in test_loader:
        inputs, root_l, vowel_l, consonant_l = data
        inputs, root_l, vowel_l, consonant_l = Variable(inputs), Variable(root_l), Variable(vowel_l), Variable(consonant_l)
        inputs, root_l, vowel_l, consonant_l = try_gpu(inputs), try_gpu(root_l), try_gpu(vowel_l), try_gpu(consonant_l)
        
        root_o, vowel_o, consonant_o = model(inputs) 
        root_pred, vowel_pred, consonant_pred = torch.max(root_o.data,1)[1], torch.max(vowel_o.data,1)[1], torch.max(consonant_o.data,1)[1]
        for i in range(inputs.size(0)):
            pred_r[index] = root_pred[i]
            pred_v[index] = vowel_pred[i]
            pred_c[index] = consonant_pred[i]
            true_r[index] = root_l[i]
            true_v[index] = vowel_l[i]
            true_c[index] = consonant_l[i]
            index += 1

    recall_r = recall_score(true_r, pred_r, average='macro')
    recall_v = recall_score(true_v, pred_v, average='macro')
    recall_c = recall_score(true_c, pred_c, average='macro')
    final_score = (2.*recall_r + recall_v + recall_c) / 4.

    print(f'Root Recall: {recall_r:.5f}')
    print(f'Vowel Recall: {recall_v:.5f}')
    print(f'Consonant Recall: {recall_c:.5f}')
    print(f'Score: {final_score:.5f}')


In [0]:
## データの読み込み

X_all = np.empty((0, HEIGHT*WIDTH))
Y_root_all = np.empty((0, 168))
Y_vowel_all = np.empty((0, 11))
Y_cons_all = np.empty((0, 7))

for parq_i in range(4):
    print(f'Parquet {parq_i} を読み込み中')
    train_df_with_img = pd.merge(pd.read_parquet(dataset_dir + f'/train_image_data_{parq_i}.parquet'), train_df, on='image_id').drop(['image_id'], axis=1)
    
    X = train_df_with_img.drop(columns=['grapheme_root', 'vowel_diacritic', 'consonant_diacritic', 'grapheme'])
    X_resized = resize(X, out_height=HEIGHT, out_width=WIDTH).astype(np.uint8) # astype(np.uint8)をしてあげることで後で cv2.cvtColor(out_data, cv2.COLOR_GRAY2RGB) が実行できるようになる
    
    Y_root = pd.get_dummies(train_df_with_img['grapheme_root']).values
    Y_vowel = pd.get_dummies(train_df_with_img['vowel_diacritic']).values
    Y_cons = pd.get_dummies(train_df_with_img['consonant_diacritic']).values

    X_all = np.append(X_all, X_resized, axis=0)
    Y_root_all = np.append(Y_root_all, Y_root, axis=0)
    Y_vowel_all = np.append(Y_vowel_all, Y_vowel, axis=0)
    Y_cons_all = np.append(Y_cons_all, Y_cons, axis=0)

    del X
    del X_resized
    del Y_root
    del Y_vowel 
    del Y_cons 
    gc.collect()

print(X_all.shape)
print(Y_root_all.shape)
print(Y_vowel_all.shape)
print(Y_cons_all.shape)

Y_all = [Y_root_all, Y_vowel_all, Y_cons_all]

trainval_dataset = MyDataset(X_all, Y_all, enable_3d=enable_3d, H=HEIGHT, W=WIDTH)

del X_all
del Y_root_all
del Y_vowel_all
del Y_cons_all
del Y_all
gc.collect()

Parquet 0 を読み込み中
Resizing raw image... / 前処理実行中…
Parquet 1 を読み込み中
Resizing raw image... / 前処理実行中…
Parquet 2 を読み込み中
Resizing raw image... / 前処理実行中…
Parquet 3 を読み込み中
Resizing raw image... / 前処理実行中…
(200840, 4096)
(200840, 168)
(200840, 11)
(200840, 7)


0

In [0]:
## 訓練ループ / Training Loop

print('==========================')
print('Starting training.')

def load_model(model, model_dir):
    device = torch.device('cpu')
    model.load_state_dict(torch.load(model_dir + '/sample_model.pth', map_location=device))
    print("model loaded from {}".format(model_dir + '/sample_model.pth'))
    model = try_gpu(model)
    return model

def save_model(model, model_dir, epoch):
    print("---saving model of epoch {}---".format(epoch))
    torch.save(model.state_dict(), model_dir + '/sample_model.pth')


if do_validation:
    n_samples = len(trainval_dataset)
    train_size = int(len(trainval_dataset)*(1.0 - val_perc))
    val_size = n_samples - train_size
    print(f'train size: {train_size}, validation size: {val_size}')

    subset, val_dataset = torch.utils.data.random_split(trainval_dataset, [train_size, val_size])

    if enable_3d:
        train_dataset = TransformDataset(subset, transform=transforms.RandomChoice(
            [transform_none, transform_crop224, transform_rotate, transform_noise]
        ))

    else:
        train_dataset = TransformDataset(subset, transform=transforms.RandomChoice(
            [transform_none, transform_crop64, transform_rotate, transform_noise]
        ))

    del trainval_dataset
    del subset
    gc.collect()

    train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, num_workers=4)
    val_loader = DataLoader(dataset=val_dataset, batch_size=32, num_workers=0)

    for i in range(start_epoch,epochs+start_epoch):
        model = load_model(model, model_dir)
        model = try_gpu(model)
        print(f'Epoch number {i}')
        train(model, i, train_loader)
        test(model, val_loader)
        save_model(model, model_dir, i)

    # メモリ節約
    del train_dataset
    del val_dataset
    del train_loader
    del val_loader
    gc.collect()

else:
    n_samples = len(trainval_dataset)
    print(f'train size: {n_samples}')

    if enable_3d:
        trainval_dataset.transform = transforms.RandomChoice(
            [transform_none, transform_crop224, transform_rotate, transform_noise]
        )
    
    else:
        trainval_dataset.transform = transforms.RandomChoice(
            [transform_none, transform_crop64, transform_rotate, transform_noise]
        )

    train_loader = DataLoader(dataset=trainval_dataset, batch_size=32, shuffle=True, num_workers=4)

    for i in range(start_epoch,epochs+start_epoch):
        model = load_model(model, model_dir)
        model = try_gpu(model)
        print(f'Epoch number {i}')
        train(model, i, train_loader)
        test(model, val_loader)
        save_model(model, model_dir, i)

    # メモリ節約
    del trainval_dataset
    del train_loader
    gc.collect()


Starting training.
train size: 160672, validation size: 40168
model loaded from ../trained_models/sample_model.pth
Epoch number 12
Root Recall: 0.94569
Vowel Recall: 0.98397
Consonant Recall: 0.97887
Score: 0.96355
---saving model of epoch 12---
model loaded from ../trained_models/sample_model.pth
Epoch number 13
Root Recall: 0.94804
Vowel Recall: 0.98279
Consonant Recall: 0.97813
Score: 0.96425
---saving model of epoch 13---
model loaded from ../trained_models/sample_model.pth
Epoch number 14
Root Recall: 0.93088
Vowel Recall: 0.97062
Consonant Recall: 0.95800
Score: 0.94759
---saving model of epoch 14---
model loaded from ../trained_models/sample_model.pth
Epoch number 15
Root Recall: 0.94166
Vowel Recall: 0.96235
Consonant Recall: 0.97113
Score: 0.95420
---saving model of epoch 15---
model loaded from ../trained_models/sample_model.pth
Epoch number 16
Root Recall: 0.93380
Vowel Recall: 0.98527
Consonant Recall: 0.97029
Score: 0.95579
---saving model of epoch 16---


In [0]:
## 提出ファイルの作成

target=[]
row_id=[] # row_id place holder

for parq_i in range(4):
    df_test_img = pd.read_parquet(dataset_dir + f'/test_image_data_{parq_i}.parquet')
    # df_test_img = pd.read_parquet(dataset_dir + f'/train_image_data_{parq_i}.parquet') # Error Check!
    df_test_img.set_index('image_id', inplace=True)

    X_test_resized = resize(df_test_img, out_height=HEIGHT, out_width=WIDTH).astype(np.uint8)

    for k, id in enumerate(df_test_img.index.values):
        X = X_test_resized[k]

        if enable_3d:
            X = cv2.resize(X.reshape(HEIGHT, WIDTH), (224, 224),interpolation=cv2.INTER_AREA)
            X = X.reshape(224, 224, 1)
            X = cv2.cvtColor(X, cv2.COLOR_GRAY2RGB)
            X = np.transpose(X, (2,0,1)) / 255.0
            X = X.reshape(1, 3, 224, 224) 
        
        else:
            X = X.reshape(1, 1, HEIGHT, WIDTH) / 255.0

        test_input = torch.tensor(X, dtype=torch.float)
        test_input = Variable(test_input)
        test_input = try_gpu(test_input)

        root_o, vowel_o, consonant_o = model(test_input)
        root_pred, vowel_pred, consonant_pred = torch.max(root_o.data,1)[1], torch.max(vowel_o.data,1)[1], torch.max(consonant_o.data,1)[1]
        if torch.cuda.is_available():
            root_pred, vowel_pred, consonant_pred = root_pred.to(torch.device("cpu")), vowel_pred.to(torch.device("cpu")), consonant_pred.to(torch.device("cpu"))

        root_pred, vowel_pred, consonant_pred = root_pred.item(), vowel_pred.item(), consonant_pred.item()
        
        row_id.append(id+'_consonant_diacritic')
        target.append(consonant_pred)
        row_id.append(id+'_grapheme_root')
        target.append(root_pred)
        row_id.append(id+'_vowel_diacritic')
        target.append(vowel_pred)
    
    del df_test_img
    del X_test_resized
    gc.collect()


df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)

if create_submission:
    df_sample.to_csv('submission.csv',index=False)

df_sample.head(36)

Resizing raw image... / 前処理実行中…
Resizing raw image... / 前処理実行中…
Resizing raw image... / 前処理実行中…
Resizing raw image... / 前処理実行中…


,row_id,target
0,Test_0_consonant_diacritic,0
1,Test_0_grapheme_root,3
2,Test_0_vowel_diacritic,1
3,Test_1_consonant_diacritic,0
4,Test_1_grapheme_root,93
5,Test_1_vowel_diacritic,2
6,Test_2_consonant_diacritic,0
7,Test_2_grapheme_root,19
8,Test_2_vowel_diacritic,0
9,Test_3_consonant_diacritic,0
